In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import edunn as nn
from edunn import utils

# Modelo de Regresión Logística: `forward`

Un modelo de Regresión Logística se forma aplicando la función `Softmax` a un modelo de Regresión Lineal. De esta forma, esta función convierte al vector de salida de la Regresión Lineal en un vector que representa una distribución de probabilidad.

La función de la Regresión logística es $f(x)=softmax(wx+b)$. No obstante, como hicimos con el modelo `LinearRegression`, podemos ver este modelo como la aplicación de
* Una capa `Linear` $f(x)=wx$,
* Una capa `Bias` $f(x)=x+b$
* Una capa `Softmax` $f(x)=softmax(x)$

Es decir, tenemos la siguiente secuencia de transformaciońes `x → Linear → Bias → Softmax → y`. 


Implementa el método `forward` del modelo `LogisticRegression` en el archivo `edunn/models/logistic_regression.py`. Para eso, ya definimos e inicializamos modelos internos de clase `Linear`, `Bias` y `Softmax`; solo tenés que llamar a sus `forward`s respectivos en el orden adecuado.

In [ ]:
x = np.array([[1,0],
             [0,1],
             [1,1]])

w = np.array([[100,0,0],
              [0,100,0],
              ])
b = np.array([0,0,0])
linear_initializer = nn.initializers.Constant(w)
bias_initializer = nn.initializers.Constant(b)
layer=nn.LogisticRegression(2,3,linear_initializer=linear_initializer,bias_initializer=bias_initializer)
y = np.array([[1, 0,0],
              [0, 1,0],
              [0.5,0.5,0]
             ])

utils.check_same(y,layer.forward(x))

y = np.array([[0, 0.5,0.5],
              [0.5, 0,0.5],
              [0,0,1]
             ])
linear_initializer = nn.initializers.Constant(-w)
bias_initializer = nn.initializers.Constant(-b)
layer=nn.LogisticRegression(2,3,linear_initializer=linear_initializer,bias_initializer=bias_initializer)
utils.check_same(y,layer.forward(x))



# Modelo de Regresión Logística: `backward`

El método `backward` de un modelo de `LogisticRegression` es la composición *inversa* de las funciones `backward` de las capas `Linear`, `Bias`, y `Softmax`. Recordá que estas se aplican en el orden contrario al método forward.

En este caso, también te ayudamos combinando el diccionario de gradientes de cada capa en un gran diccionario único de gradientes de `LogisticRegression` utilizando el operador `**` que desarma un diccionario, con `{**dict1, **dict2}` que los vuelve a combinar.

Implementá el método `backward` del modelo `LogisticRegression`:


In [ ]:
samples = 100
batch_size=2
din=3 # dimensión de entrada
dout=5 # dimensión de salida
input_shape=(batch_size,din)

# Verificar las derivadas de un modelo de Regresión Logística
# con valores aleatorios de `w`, `b`, y `x`, la entrada
layer=nn.LogisticRegression(din,dout)

utils.check_gradient.common_layer(layer,input_shape,samples=samples,tolerance=1e-5)    


# Regresión Logística aplicada

Ahora que tenemos todos los elementos, podemos definir y entrenar nuestro primer modelo regresión logística para clasificar las flores del conjunto de datos de [Iris](https://www.kaggle.com/uciml/iris).

En este caso, vamos a entrenar el modelo con la función de error cuadrático medio; no obstante, si bien esta forma del error funcionará para este problema, hace el que problema de optimización no sea _convexo_ y por ende no haya un único mínimo global. Más adelante, implementaremos la función de error de _Entropía Cruzada_, diseñada específicamente para lidiar con salidas que representan distribuciones de probabilidad.

In [ ]:
import edunn as nn
import numpy as np
from edunn import metrics,datasets

# Cargar datos, donde `y` tiene codificación "onehot"
# `y` tiene tantas columnas como clases
# si el ejemplo i es de clase 2, por ejemplo, entonces
# y[i,2]=1 y el resto de los valores de y[i,:] valen 0
# (nota: las etiquetas de clase comienzan en 0)
x,y,classes=datasets.load_classification("iris",onehot=True)
# normalización de los datos
x = (x-x.mean(axis=0))/x.std(axis=0)
n, din = x.shape
n, dout = y.shape

print("Dataset sizes:", x.shape,y.shape)

#Modelo de regresión logística
model = nn.LogisticRegression(din,dout)
# Error cuadrático medio
error = nn.MeanError(nn.SquaredError())
optimizer = nn.GradientDescent(lr=0.1,epochs=1000,batch_size=32)

# Algoritmo de optimización
history = optimizer.optimize(model,x,y,error)
nn.plot.plot_history(history,error_name=error.name)


print("Error del modelo:")
y_pred=model.forward(x)
metrics.classification_summary_onehot(y,y_pred)
